In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from birds_utils import Dataset, get_pytorch_model_mel, train_model
import torch

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
train_config ={
    'dataset': './dataset/bin_22050_32/',
    'file_stats': './files_std_data_32_bits.npy',
    'duration': 5,
    'SAMPLE_RATE': 22050,
    'pretrained_model': 'resnet18',
    'pretrained': True,
    'n_classes': 264,
    'n_mels': 128,
    'window_size': 2048,
    'train_batch_size': 32,
    'val_batch_size': 32,
    'optimizer': 'Adam',
    'LEARNING_RATE': 0.001,
    'classes_file': './models/classes_264.npy',
    'train_files': './train_files_22050_264_32.npy',
    'val_files': './val_files_22050_264_32.npy',
    'N_samples_train': 2,
    'N_samples_val': 4,
    'f1_kaggle': '',
    'last_training_state': './models/librosa_model_train_state_mel_128_264_32.pth',
    'best_model': './models/model_mel_264_v2_32.pth',
    'init_fourier': True,
    'init_mel': True,
    'train_fourier': False,
    'train_mel': True,
    'train_add_noise': True,
    'train_add_ambient_noise': True,
    'val_add_noise': False,
    'val_add_ambient_noise': False,
    'bytes_per_sample': 4
}

In [5]:
model = get_pytorch_model_mel(
    window_size=train_config['window_size'], 
    resnet=train_config['pretrained_model'], 
    pretrained=train_config['pretrained'],
    n_classes=train_config['n_classes'], 
    init_fourier=train_config['init_fourier'], 
    init_mel=train_config['init_mel'], 
    train_fourier=train_config['train_fourier'], 
    n_mels=train_config['n_mels'],
    sr=train_config['SAMPLE_RATE']

).to(device)

Using cache found in /home/usuario/.cache/torch/hub/pytorch_vision_v0.6.0


In [6]:
classes = np.load(train_config['classes_file'])
file_stats = np.load(train_config['file_stats'], allow_pickle=True).item()
train_files = np.load(train_config['train_files'])
val_files = np.load(train_config['val_files'])

In [7]:
val_files_large = []
for file in val_files:
    if file_stats[file]['size'] >= train_config['SAMPLE_RATE']*train_config['duration']:
        val_files_large.append(file)

In [8]:
params_train = {
    'batch_size': train_config['train_batch_size'],
    'shuffle': True,
    'num_workers': 0
}

params_val = {
    'batch_size': train_config['val_batch_size'],
    'shuffle': True,
    'num_workers': 0,
    'timeout': 0
}


training_set = Dataset(
    list_IDs=list(train_files)*train_config['N_samples_train'], 
    classes=classes, 
    std_stats=file_stats, 
    chunk_seconds=train_config['duration'], 
    bytes_per_sample=train_config['bytes_per_sample'],
    sr=train_config['SAMPLE_RATE'], 
    add_noise=train_config['train_add_noise'], 
    add_ambient_noise=train_config['train_add_ambient_noise']
)

validation_set = Dataset(
    list_IDs=val_files_large*train_config['N_samples_val'], 
    classes=classes, 
    std_stats=file_stats, 
    chunk_seconds=train_config['duration'], 
    bytes_per_sample=train_config['bytes_per_sample'],
    sr=train_config['SAMPLE_RATE'],
    add_noise=train_config['val_add_noise'], 
    add_ambient_noise=train_config['val_add_ambient_noise']
)
training_generator = torch.utils.data.DataLoader(training_set, **params_train)
validation_generator = torch.utils.data.DataLoader(validation_set, **params_val)

In [9]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=train_config['LEARNING_RATE'])

In [ ]:
n_classes = train_config['n_classes']

epochs, losses, F1s, val_losses, val_F1s = train_model(
    model, 
    training_generator, 
    validation_generator, 
    criterion=criterion, 
    optimizer=optimizer,
    name=f'./models/model_mel_v2_128_32_{n_classes}', 
    device=device,
    epochs=25
)

1/25 - 1067/1067 - loss: 0.02317820116877556, F1 micro: 0.009694719687104225, acc: 0.005826747044920921, elapesed_time: 0:14:52.994340
506/506 - val loss: 0.01929677277803421, val F1 micro: 0.07048851251602173 val acc: 0.0390412658452987735
Best model saved
--------------------------------------------------------------------------
2/25 - 1067/1067 - loss: 0.01603003963828087, F1 micro: 0.12391232699155807, acc: 0.06974365562200546, elapesed_time: 0:15:01.384294
506/506 - val loss: 0.016249969601631165, val F1 micro: 0.23465649783611298 val acc: 0.14844329655170442
Best model saved
--------------------------------------------------------------------------
3/25 - 1067/1067 - loss: 0.01315353810787201, F1 micro: 0.2978919446468353, acc: 0.1903521865606308, elapesed_time: 0:16:09.113058
506/506 - val loss: 0.013348674401640892, val F1 micro: 0.39563482999801636 val acc: 0.29114159941673286
Best model saved
--------------------------------------------------------------------------
4/25 - 10

In [44]:
train_config['last_training_state']
train_config['best_model'] = 'model_mel_v2_264_0.00943.pth'


In [43]:
torch.save({
    'epoch': 25,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, train_config['last_training_state'])

In [41]:
train_config['results'] = {}
train_config['results']['losses'] = losses_d
train_config['results']['val_losses'] = val_losses
train_config['results']['f1_score'] = F1s_d
train_config['results']['val_f1_score'] = val_F1s

In [45]:
import time
np.save(f'training_configs/train_config_{int(time.time())}_mel_no_train.npy', train_config)

In [46]:
print(training_set.stats)
print(validation_set.stats)

{'noise': 0, 'noise+signal': 0, 'signal': 0, 'passband_noise': 0, 'short_file': 0}
{'noise': 22700, 'noise+signal': 0, 'signal': 404700, 'passband_noise': 0, 'short_file': 22700}


In [ ]:
# sd